### Imports:

In [1]:
from taxonomy_ranks import TaxonomyRanks
import numpy as np
import pandas as pd

Load the cleaned data and extract the taxa names:

In [2]:
data = pd.read_csv("../data_cleaned.csv")
taxa_names = data.columns[7:].values.copy()

### Inspection Script:
Inspects the taxa names. Specifically looking for:
- ambiguous IDs
- missing names

In [3]:
ranks = ('phylum', 'class', 'order', 'family', 'genus')

def inspect_taxa_names(taxa_names):
    """
    Inspect the taxa names for potential issues such as ambiguous IDs, missing names, or misspellings.

    Parameters:
    taxa_names (array-like): An array of taxa names to inspect.

    Returns:
    np.ndarray: An array containing the lineage data for the taxa names.
    """
    
    # prepare a counter and list for ambiguous ids
    several_ids_counter = 0
    several_ids_names = []

    # prepare a counter and lists for missing taxa
    missing_counter = 0
    missing_names = []
    missing_indices = []

    # empty array prepared for final data
    my_data = np.array([])

    # loop through all taxa names, find their lineage, fill taxa_names_cleaned with lineage
    for i in range(len(taxa_names)):
        rank_taxon = TaxonomyRanks(taxa_names[i])
        
        # if it doesn't find the taxid it throws an error, in which case we catch it
        # and remember the taxon name to look it up online later
        try: 
            # find all ids and names
            rank_taxon.get_lineage_taxids_and_taxanames()
            
            # remember those taxa names for which several ids were found
            potential_taxids = rank_taxon.potential_taxids
            if len(potential_taxids) > 1:
                several_ids_counter += 1
                several_ids_names.append(taxa_names[i])
            
            # add the current ranks to the final data
            curr_data = []
            for j, rank in enumerate(ranks):
                rank_name = rank_taxon.lineages[str(potential_taxids[0])][rank]
                curr_data.append(rank_name[0])
            curr_data = np.array(curr_data)[np.newaxis, :]
            if my_data.size == 0:
                my_data = curr_data
            else: 
                my_data = np.concatenate([my_data,curr_data], axis=0)

        except:
            # if the taxon wasn't found, remember it to later look it up online
            missing_counter += 1
            missing_names.append(taxa_names[i])
            missing_indices.append(i)


    print("\n", several_ids_counter,
        " of the ", len(taxa_names), " taxa had several ids.")
    print("Namely: ", several_ids_names)

    print("\n Couldn't find: ", missing_counter,
        " of the ", len(taxa_names), " taxa.")
    print("Namely: ", missing_names)
    print("At indices: ", missing_indices)


    print("\n Make sure that these dimensions match!")
    print("my_data.shape: ", my_data.shape[0])
    print("taxa_names.shape: ", taxa_names.shape[0])

    return my_data

In [4]:
_ = inspect_taxa_names(taxa_names)


 5  of the  130  taxa had several ids.
Namely:  ['Bacillus', 'Fusobacteria', 'Proteus et rel.', 'Serratia', 'Yersinia et rel.']

 Couldn't find:  10  of the  130  taxa.
Namely:  ['Allistipes et rel.', 'Klebisiella pneumoniae et rel.', 'Lachnobacillus bovis et rel.', 'Outgrouping clostridium cluster XIVa', 'Uncultured Chroococcales', 'Uncultured Clostridium (sensu stricto)les I', 'Uncultured Clostridium (sensu stricto)les II', 'Uncultured Mollicutes', 'Uncultured Selenomonadaceae', 'Wissella et rel.']
At indices:  [5, 70, 71, 88, 119, 120, 121, 122, 123, 127]

 Make sure that these dimensions match!
my_data.shape:  120
taxa_names.shape:  130


### Manual Inspection:
I compared both the ambiguous and the missing IDs via https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi
1. I looked up the ambiguous IDs ('Bacillus', 'Fusobacteria', 'Proteus et rel.', 'Serratia', 'Yersinia et rel.'). I found the ambiguity to derive from large animals with similar rank names, the default ID (0) however is right in all cases.

2. The missing names were misspelled, had unclear assignments, or were names of higher ranks with no genus in the database. I changed them in the following by manual imputation:

In [5]:
# misspelled:
taxa_names[5] = "Alistipes"
taxa_names[70] = "Klebsiella pneumoniae"
taxa_names[71] = "Lachnobacterium bovis"
taxa_names[119] = "uncultured Chlorococcales"
taxa_names[127] = "Weissella"
# The following: 'Outgrouping clostridium cluster XIVa','Uncultured Clostridium (sensu stricto)les I', 'Uncultured Clostridium (sensu stricto)les II'
# had no match. I decided to replace them all by "Clostridium". This leaves us with redundant naming, which however is no problem.
taxa_names[88] = "Clostridium"
taxa_names[120] = "Clostridium"
taxa_names[121] = "Clostridium"
# The following are names of higher ranks with no genus
taxa_names[122] = "Mollicutes" # original: "Uncultured Mollicutes" - I imputed from the class rank
taxa_names[123] = "Selenomonadaceae" # original: "Uncultured Selenomonodaceae" - I imputed from the family rank


In [6]:
taxa_names_cleaned = inspect_taxa_names(taxa_names)


 5  of the  130  taxa had several ids.
Namely:  ['Bacillus', 'Fusobacteria', 'Proteus et rel.', 'Serratia', 'Yersinia et rel.']

 Couldn't find:  0  of the  130  taxa.
Namely:  []
At indices:  []

 Make sure that these dimensions match!
my_data.shape:  130
taxa_names.shape:  130


The kingdom column was entirely empty, so it was omitted from ranks.\
Species were never recognized, so I imputed the original uncorrected names for compatibility 
with the main dataset (data_cleaned.csv).\
The following ranks were used: **phylum, class, order, family, genus, species**.

In [7]:
ranks = list(ranks)
ranks.append("species")
species = data.columns[7:].values[:, np.newaxis]
taxa_names_cleaned = np.hstack([taxa_names_cleaned,species])

The following code checks in which rows / for which bacteria some of the ranks are missing:

In [8]:
def incomplete_rows(input_data):
    """
    Check which rows in the input data are incomplete (i.e., contain 'NA').

    Parameters:
    input_data (np.ndarray): The input data array to check for incomplete rows.

    Returns:
    None
    """
    input_data = input_data.copy()
    incomplete_rows = np.unique(np.where(input_data == 'NA')[0])
    print("Indices of incomplete rows: ", incomplete_rows)
    print("Incomplete rows: ", input_data[incomplete_rows])

incomplete_rows(taxa_names_cleaned)

Indices of incomplete rows:  [  0  12  65  82  84 118 119 122 123 128]
Incomplete rows:  [['Actinomycetota' 'Actinomycetes' 'Actinomycetales' 'Actinomycetaceae'
  'NA' 'Actinomycetaceae']
 ['Pseudomonadota' 'Betaproteobacteria' 'Burkholderiales' 'NA'
  'Aquabacterium' 'Aquabacterium']
 ['Fusobacteriota' 'NA' 'NA' 'NA' 'NA' 'Fusobacteria']
 ['Actinomycetota' 'Actinomycetes' 'Micrococcales' 'Micrococcaceae' 'NA'
  'Micrococcaceae']
 ['Pseudomonadota' 'Gammaproteobacteria' 'Moraxellales' 'Moraxellaceae'
  'NA' 'Moraxellaceae']
 ['Bacteroidota' 'NA' 'NA' 'NA' 'NA' 'Uncultured Bacteroidetes']
 ['Chlorophyta' 'Chlorophyceae' 'Chlamydomonadales' 'NA' 'NA'
  'Uncultured Chroococcales']
 ['Mycoplasmatota' 'Mollicutes' 'NA' 'NA' 'NA' 'Uncultured Mollicutes']
 ['Bacillota' 'Negativicutes' 'Selenomonadales' 'Selenomonadaceae' 'NA'
  'Uncultured Selenomonadaceae']
 ['Pseudomonadota' 'Gammaproteobacteria' 'Lysobacterales'
  'Lysobacteraceae' 'NA' 'Xanthomonadaceae']]


Missing Rank Imputation:\
For any row that some rank is missing, I manually impute the missing rank with the previous (not missing) rank:

In [9]:
# impute genus with family
taxa_names_cleaned[0][4] = taxa_names_cleaned[0][3]
taxa_names_cleaned[82][4] = taxa_names_cleaned[82][3]
taxa_names_cleaned[84][4] = taxa_names_cleaned[84][3]
taxa_names_cleaned[123][4] = taxa_names_cleaned[123][3]
taxa_names_cleaned[128][4] = taxa_names_cleaned[128][3]
#impute family with genus
taxa_names_cleaned[12][3] = taxa_names_cleaned[12][4]
taxa_names_cleaned[66][3] = taxa_names_cleaned[66][4]
# impute everything with the phylum
taxa_names_cleaned[65][1] = taxa_names_cleaned[65][0]
taxa_names_cleaned[65][2] = taxa_names_cleaned[65][0]
taxa_names_cleaned[65][3] = taxa_names_cleaned[65][0]
taxa_names_cleaned[65][4] = taxa_names_cleaned[65][0]
taxa_names_cleaned[118][1] = taxa_names_cleaned[118][0]
taxa_names_cleaned[118][2] = taxa_names_cleaned[118][0]
taxa_names_cleaned[118][3] = taxa_names_cleaned[118][0]
taxa_names_cleaned[118][4] = taxa_names_cleaned[118][0]
# impute family and genus with order
taxa_names_cleaned[119][3] = taxa_names_cleaned[119][2]
taxa_names_cleaned[119][4] = taxa_names_cleaned[119][2]
#impute order,family and genus with class
taxa_names_cleaned[122][2] = taxa_names_cleaned[122][1]
taxa_names_cleaned[122][3] = taxa_names_cleaned[122][1]
taxa_names_cleaned[122][4] = taxa_names_cleaned[122][1]


# print how many categories of each rank are present in the data
for i, rank in enumerate(ranks):
    print("\n", rank, " has ", len(
        np.unique(taxa_names_cleaned[:, i])), " unique columns.")

# check if there are any incomplete rows left
incomplete_rows(taxa_names_cleaned)


 phylum  has  11  unique columns.

 class  has  18  unique columns.

 order  has  37  unique columns.

 family  has  62  unique columns.

 genus  has  88  unique columns.

 species  has  130  unique columns.
Indices of incomplete rows:  []
Incomplete rows:  []


### Export the final lineage dataset and update data_cleaned with these changes:

In [10]:
# If all checks are passed, we can merge the data with the ranks
lineage_data = pd.DataFrame(data=taxa_names_cleaned, columns = ranks)
print(lineage_data.head())

# export it as csv
# lineage_data.to_csv("lineage_data.csv", sep=',')

              phylum                class               order  \
0     Actinomycetota        Actinomycetes     Actinomycetales   
1          Bacillota              Bacilli     Lactobacillales   
2     Pseudomonadota  Gammaproteobacteria       Aeromonadales   
3  Verrucomicrobiota     Verrucomicrobiia  Verrucomicrobiales   
4     Pseudomonadota   Betaproteobacteria     Burkholderiales   

             family             genus                       species  
0  Actinomycetaceae  Actinomycetaceae              Actinomycetaceae  
1     Aerococcaceae        Aerococcus                    Aerococcus  
2    Aeromonadaceae         Aeromonas                     Aeromonas  
3   Akkermansiaceae       Akkermansia                   Akkermansia  
4    Alcaligenaceae       Alcaligenes  Alcaligenes faecalis et rel.  


In [11]:
# Change the names in the main dataset as well:
data.columns = np.hstack([data.columns[:7], taxa_names_cleaned[:, 4]]) 
print(data.head())

# export to data_cleaned.csv
# data.to_csv("../data_cleaned.csv", sep=',', index=False)

   index  SampleID   Age     Sex Nationality  Diversity    BMI_group  \
0      0  Sample-1  28.0    male          US       5.76  severeobese   
1      1  Sample-2  24.0  female          US       6.06        obese   
2      2  Sample-3  52.0    male          US       5.50         lean   
3      3  Sample-4  22.0  female          US       5.87  underweight   
4      4  Sample-5  25.0  female          US       5.89         lean   

   Actinomycetaceae  Aerococcus  Aeromonas  ...   Clostridium   Clostridium  \
0         72.018955   36.677499  42.602327  ...   1104.414062   1028.468474   
1         71.617440   35.093902  38.339812  ...  18897.145365  10796.271198   
2         76.788552   36.468459  41.049680  ...   9941.426890   2382.601913   
3         74.153361   35.983472  40.919850  ...   1707.346473   2881.873156   
4         77.065064   37.466006  42.746629  ...  32454.018243   7598.565773   

     Mollicutes  Selenomonadaceae  Veillonella      Vibrio   Weissella  \
0    521.322821   